# Extract sources from catalogs  in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/18
- update : 2024/02/24


         DatasetType('isolated_star_cat', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+34187316c4 	current w_2024_07 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [12]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Sources

In [13]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe

#'isolated_star_cat', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),

data_product = "fgcmVisitCatalog"

#[DatasetType('fgcmFitParameters0', {instrument}, Catalog),
# DatasetType('fgcmMakeLut_config', {}, Config),
# DatasetType('fgcm_standard_passband', {band, instrument, physical_filter}, TransmissionCurve),
# DatasetType('fgcm_standard_atmosphere', {instrument}, TransmissionCurve),
# DatasetType('fgcmLookUpTable', {instrument}, Catalog),
# DatasetType('fgcmMakeLut_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmMakeLut_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmBuildFromIsolatedStars_config', {}, Config),
# DatasetType('fgcm_star_ids', {instrument}, ArrowAstropy),
# DatasetType('fgcm_reference_stars', {instrument}, ArrowAstropy),
# DatasetType('fgcmVisitCatalog', {instrument}, Catalog),
# DatasetType('fgcm_star_observations', {instrument}, ArrowAstropy),
# DatasetType('fgcmBuildFromIsolatedStars_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmBuildFromIsolatedStars_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmFitCycle_config', {}, Config),
# DatasetType('fgcmFlaggedStars0', {instrument}, Catalog),
# DatasetType('fgcmAtmosphereParameters4', {instrument}, Catalog),
# DatasetType('fgcmFitParameters4', {instrument}, Catalog),
# DatasetType('fgcmZeropoints4', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars1', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars2', {instrument}, Catalog),
# DatasetType('fgcmStandardStars4', {instrument}, SimpleCatalog),
# DatasetType('fgcmFitParameters3', {instrument}, Catalog),
# DatasetType('fgcmFitParameters1', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars4', {instrument}, Catalog),
# DatasetType('fgcmFitParameters2', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars3', {instrument}, Catalog),
# DatasetType('fgcmFitCycle_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmFitCycle_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmStandardStars6', {instrument}, SimpleCatalog),
# DatasetType('fgcmZeropoints6', {instrument}, Catalog),
# DatasetType('fgcmFitParameters5', {instrument}, Catalog),
# DatasetType('fgcmAtmosphereParameters6', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars6', {instrument}, Catalog),
# DatasetType('fgcmFitParameters6', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars5', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars8', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars7', {instrument}, Catalog),
# DatasetType('fgcmFitParameters8', {instrument}, Catalog),
# DatasetType('fgcmFitParameters9', {instrument}, Catalog),
# DatasetType('fgcmFitParameters7', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars9', {instrument}, Catalog),
# DatasetType('fgcmOutputProducts_metadata', {instrument}, TaskMetadata),
# DatasetType('transmission_atmosphere_fgcm', {band, instrument, physical_filter, visit}, TransmissionCurve),
# DatasetType('fgcmPhotoCalibCatalog', {band, instrument, physical_filter, visit}, ExposureCatalog),
# DatasetType('fgcmOutputProducts_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmOutputProducts_config', {}, Config)]


datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [31]:
all_df = []

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
    
    data = butler.get(ref)
    t = data.asAstropy()
    print(t)
   
    if(len(df)>0):
        print(type(df),len(df))
      
        all_df.append(df)
    

========================datasetType = fgcmVisitCatalog ============================================
fullId.............................: {instrument: 'LATISS'}
    visit      physicalFilter        telra        ... scaling used sources_read
------------- ---------------- ------------------ ... ------- ---- ------------
2023051100262 SDSSr_65mm~empty   239.941279369513 ...     1.0    1        False
2023051100263 SDSSg_65mm~empty 239.94309908614096 ...     1.0    1        False
2023051100264 SDSSi_65mm~empty 239.94091850747296 ...     1.0    1        False
2023051100268 SDSSr_65mm~empty 239.96007906570497 ...     1.0    1        False
2023051100269 SDSSg_65mm~empty   239.959844616716 ...     1.0    1        False
2023051100270 SDSSi_65mm~empty 239.96385047378996 ...     1.0    1        False
2023051100271 SDSSi_65mm~empty   239.973089638681 ...     1.0    1        False
2023051100272 SDSSg_65mm~empty   239.976800668184 ...     1.0    1        False
2023051100273 SDSSr_65mm~empty   239.983

/tmp/ipykernel_22346/2411115195.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


In [29]:
t.to_pandas()

ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['scaling']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)

In [27]:
t.to_pandas()

ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['scaling']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)

In [25]:
df = pd.concat(all_df)

TypeError: cannot concatenate object of type '<class 'lsst.afw.table.BaseCatalog'>'; only Series and DataFrame objs are valid

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())

In [ ]:
df['tract'].unique()